## Ensure a Minikube cluster is running

- If you created a Minikube cluster with the command `minikube start --nodes 3` and it is still running:
  - Do nothing
- If you have a stopped (not deleted) Minikube cluster that was created with the command `minikube start --nodes 3`:
  - Run the command `minikube start`
- If you don't have a Minikube cluster (or deleted your previous one):
  - Run the command `minikube start --nodes 3`  

## Deploy a Service

- The Service definition is in the YAML file `manifets/clusterip.yaml`

In [1]:
!kubectl apply -f manifests/clusterip.yaml

service/svc-example created


## Let's look at the Service's YAML

**Note:**

- The service's name is `svc-example`.
- The service's is:
  - Listening on `port` 8080.
  - Redirecting traffic to `targetPort` 80.
- The servie's `selector` defines two labels:
  - `app: app-example` and `env: prod`

```bash
apiVersion: v1
kind: Service
metadata:
  name: svc-example   # the Service's name
spec:
  ports:
  - port: 8080        # the Service is listening on port 8080
    targetPort: 80    # the Service is redirecting traffic to targetPort 80
  selector:
    app: app-example  # the Service's selector defines two labels
    env: prod         # app = app-example and env = prod
```

In [2]:
!type manifests/clusterip.yaml # use this on Windows
!cat manifests/clusterip.yaml

/bin/bash: line 0: type: manifests/clusterip.yaml: not found
apiVersion: v1
kind: Service
metadata:
  name: svc-example
spec:
  ports:
  - port: 8080
    targetPort: 80
  selector:
    app: app-example
    env: prod

## Deploy a Deployment

-The Deployment definition is in the YAML file `manifests/deploy-app.yaml`.

In [3]:
!kubectl apply -f manifests/deploy-app.yaml

deployment.apps/deploy-example created


## Let's look at the Deployment's YAML

**Note:**

- The number of `replicas` is set to 3.
- The `revisionHistoryLimit` is set to 3.
- The Pod template's container is listening on `containerPort` 80.
- The Pod template's `labels` are `app: app-example` and `type: prod`
  - The Deployment's `matchLabels` match these labels
    - Therefore, the Deployment will create 3 replicas from the Pod template.
  - The Service's `selector` matches these labels
    - Therefore, the Service will load balance between the three Pod replicas.

```bash
apiVersion: apps/v1
kind: Deployment
metadata:
  name: deploy-example        # the deployment is named deploy-example
spec:
  replicas: 3                 # the deployment defines three replicas
  revisionHistoryLimit: 3     # the deployment defines a maximum of three ReplicaSet revisions to store in history
  selector:
    matchLabels:
      app: app-example        # the deployment has two matchLabels that match the Pod template's labels below
      type: prod              # app = app-example and type = prod
  template:
    metadata:
      name: myapp-pod
      labels:
        app: app-example       # the Pod template has two labels defined (the Service's selector matches these two labels):
        type: prod             # app = myapp and type = front-end
    spec:
      containers:
      - name: nginx
        image: nginx:alpine    # the container is based onthe nginx:alpine image
        resources:
          requests:
            cpu: 100m
            memory: 128Mi
          limits:
            cpu: 250m
            memory: 256Mi    
        ports:
        - containerPort: 80    # the Pod template's container is listening on containerPort 80
```

In [4]:
#!type manifests/deploy-app.yaml # use this on Windows
!cat manifests/deploy-app.yaml

apiVersion: apps/v1
kind: Deployment
metadata:
  name: deploy-example
spec:
  replicas: 3
  revisionHistoryLimit: 3
  selector:
    matchLabels:
      app: app-example
      env: prod
  template:
    metadata:
      labels:
        app: app-example
        env: prod
    spec:
      containers:
      - name: nginx
        image: nginx:alpine
        resources:
          requests:
            cpu: 100m
            memory: 128Mi
          limits:
            cpu: 250m
            memory: 256Mi
        ports:
        - containerPort: 80

## Deploy a Busybox Pod

- The Pod definition is in the YAML file `manifests/pod.yaml`.

In [5]:
!kubectl apply -f manifests/pod.yaml

pod/mybox created


## Let's look at the BusyBox Pos's YAML

**Note:**

- The Pods name is `mybox`.
- The container's name is `mybox`.
- The container is based on the `busybox` image.
  - This image contains numerous useful tools pre-installed.
- The container executes the command `sleep 3600` when it starts.
  - This is jsut to give us enough time (3600 seconds) to remote in to it before it terminates.

```bash
apiVersion: v1
kind: Pod
metadata:
  name: mybox              # the Pod's name is mybox
spec:
  restartPolicy: Always
  containers:
  - name: mybox            # the containers's name is mybox
    image: busybox         # the containers is based on the image busybox
    resources:
      requests:
        cpu: 100m
        memory: 128Mi
      limits:
        cpu: 250m
        memory: 256Mi    
    command:                # the container executes the command 'sleep 3600' when it starts
      - sleep
      - "3600"
```

In [6]:
#!type manifests/pod.yaml #use this on Windows
!cat manifests/pod.yaml

apiVersion: v1
kind: Pod
metadata:
  name: mybox
spec:
  restartPolicy: Always
  containers:
  - name: mybox
    image: busybox
    resources:
      requests:
        cpu: 100m
        memory: 128Mi
      limits:
        cpu: 250m
        memory: 256Mi    
    command:
      - sleep
      - "3600"

## List Pods

- We see that 4 Pods are running:
  - 3 Pods, that were defined in the Deployment, are running containers based on the `nginx:alpine` image.
  - 1 Pod is running a container based on the `nginx:busybox` image.

In [7]:
#!kubectl get po -o wide
!kubectl get pods -o wide

NAME                              READY   STATUS    RESTARTS   AGE   IP           NODE           NOMINATED NODE   READINESS GATES
deploy-example-7b9798d5cf-dvllc   1/1     Running   0          24s   10.244.0.8   minikube       <none>           <none>
deploy-example-7b9798d5cf-krh9n   1/1     Running   0          24s   10.244.2.6   minikube-m03   <none>           <none>
deploy-example-7b9798d5cf-vwmnb   1/1     Running   0          24s   10.244.1.6   minikube-m02   <none>           <none>
mybox                             1/1     Running   0          14s   10.244.1.7   minikube-m02   <none>           <none>


## List Services

- Notice the `TYPE` is `ClusterIP` for the `svc-example` Service.
  - This means that the Service has a private IP address that can only be accessed within the cluster.
  - This is the default setting for a Service if no service type is explicitly defined.
- We also see that the Service is listening on port 8080.

In [8]:
#!kubectl get svc -o wide
!kubectl get services -o wide

NAME          TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)    AGE   SELECTOR
kubernetes    ClusterIP   10.96.0.1       <none>        443/TCP    11h   <none>
svc-example   ClusterIP   10.111.64.120   <none>        8080/TCP   42s   app=app-example,env=prod


## Check the Service Endpoint

- Notice the Service's `ENDPOINTS` contains three IP addresses.
  - The service's `selector` finds three Pods with the labels `app: app-example` and `type: prod`.
  - This is because the Pod template's labels in the Deployment are `app: app-example` and `type: prod`.
- The Service will (round robin) load balance between these three IP addresses.

In [9]:
#!kubectl get ep svc-example
!kubectl get endpoints svc-example

NAME          ENDPOINTS                                   AGE
svc-example   10.244.0.8:80,10.244.1.6:80,10.244.2.6:80   58s


## Change the Nginx default web page in the Pod's containers

- Let's set the default Nginx web page for
  - the first Pod's container to `Hello World 1`
  - the second Pod's container to `Hello World 2`.
  - the third Pod's container to `Hello World 3`.
- Now we should be able to see the Service (round robin) load balancing between the three Pod replicas.

In [11]:
POD_NAME_1 = !kubectl get pods -o jsonpath='{.items[0].metadata.name}'
POD_NAME_2 = !kubectl get pods -o jsonpath='{.items[1].metadata.name}'
POD_NAME_3 = !kubectl get pods -o jsonpath='{.items[2].metadata.name}'
POD_NAME_1 = POD_NAME_1[0]
POD_NAME_2 = POD_NAME_2[0]
POD_NAME_3 = POD_NAME_3[0]
!kubectl exec {POD_NAME_1} -- sh -c "echo 'Hello World 1' > /usr/share/nginx/html/index.html"
!kubectl exec {POD_NAME_2} -- sh -c "echo 'Hello World 2' > /usr/share/nginx/html/index.html"
!kubectl exec {POD_NAME_3} -- sh -c "echo 'Hello World 3' > /usr/share/nginx/html/index.html"

## Get Nginx default web page through service via the BusyBox container

- Traffic Flow:  BusyBox Pod's container (80) <--> Service (8080) <--> (round robin load balancing) Deployment Pod's Container (80).
- The Deployment Pod's containers are running an nginx web server on port 80 (that serves the default web page).

### Run these commands in a terminal (won't work from a notebook cell)

```bash
# Connect to the BusyBox container via an interactive session
kubectl exec mybox -it -- /bin/sh

# --- output ---
# / #
# --------------

# Get the Nginx home page through the ClusterIP service called "svc-example" listening on port 8080
wget -qO- http://svc-example:8080

# --- output ---
# Hello World 1
# --------------

# Get the Nginx home page through the ClusterIP service again
wget -qO- http://svc-example:8080

# --- output ---
# Hello World 2
# --------------

# Get the Nginx home page through the ClusterIP service again
wget -qO- http://svc-example:8080

# --- output ---
# Hello World 3
# --------------

# Get the Nginx home page through the ClusterIP service again
wget -qO- http://svc-example:8080

# --- output ---
# Hello World 1
# --------------

# Exit the interactive session
exit

# --- output ---
# -
# --------------
```

## Delete Service, Deployment and BusyBox Pod

In [12]:
!kubectl delete -f manifests/clusterip.yaml
!kubectl delete -f manifests/deploy-app.yaml
!kubectl delete -f manifests/pod.yaml --grace-period=0

service "svc-example" deleted
deployment.apps "deploy-example" deleted
pod "mybox" deleted


## List Services, Deployments and Pods

- We see that the Service, Deployment and Pods have been deleted.

In [13]:
#!kubectl get svc
#!kubectl get deploy
#!kubectl get po
!kubectl get services
!kubectl get deployments
!kubectl get pods

NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   11h
No resources found in default namespace.
No resources found in default namespace.
